SVMwoEdge.ipynb by:

Lloyd Fernandes (lloydf2@illinois.edu)

Project director: Richard Sowers r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/ Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

In [ ]:
import pandas as pd
import numpy as np
from data import *
from feature_eng import *

In [53]:
df = pd.read_pickle("sample_features.pkl")
#df = df.droplevel('traj')
df.head()

lat        lon    speed  lon_acc  lat_acc type  \
id traj time                                                         
12 0    0.00  37.977641  23.735304  35.2628  -0.0473   0.0150  Bus   
        0.04  37.977644  23.735301  35.2602   0.0113   0.0172  Bus   
        0.08  37.977647  23.735299  35.2644   0.0473   0.0194  Bus   
        0.12  37.977650  23.735296  35.2738   0.0833   0.0216  Bus   
        0.16  37.977653  23.735294  35.2909   0.1531   0.0238  Bus   

              traveled_d  avg_speed   bearing  nearest_edge_start_node  \
id traj time                                                             
12 0    0.00       296.5  15.931194 -0.170808                250691723   
        0.04       296.5  15.931194 -0.114489                250691723   
        0.08       296.5  15.931194 -0.170804                250691723   
        0.12       296.5  15.931194 -0.114485                250691723   
        0.16       296.5  15.931194 -0.170799                250691723   

              nearest_edge_end_node  edge_progress  dir  time_stamp  \
id traj time                                                          
12 0    0.00             3339821648       0.390833    0        0.00   
        0.04             3339821648       0.388197    0        0.04   
        0.08             3339821648       0.385899    0        0.08   
        0.12             3339821648       0.383263    0        0.12   
        0.16             3339821648       0.380967    0        0.16   

              edge_progress_intervals  vehicle_density  avg_surr_speed  \
id traj time                                                             
12 0    0.00                      0.3                5        34.89136   
        0.04                      0.3                5        34.89094   
        0.08                      0.3                5        34.89164   
        0.12                      0.3                5        34.89320   
        0.16                      0.3                5        34.89608   

                           edge_id  
id traj time                        
12 0    0.00  250691723_3339821648  
        0.04  250691723_3339821648  
        0.08  250691723_3339821648  
        0.12  250691723_3339821648  
        0.16  250691723_3339821648

In [54]:
#remove all vehicles except taxi and car
df = df[(df['type'] == 'Car' )|( df['type'] == 'Taxi')]

#to one-hot-encode edges
# df_edge_list = df.reset_index()[['id','edge_id']].drop_duplicates()
# df_edge_list.set_index(['id','edge_id'],inplace = True)
# df_edge_list['edge_id'] = df_edge_list.index.get_level_values('edge_id') 
# df_edge_dummies = pd.get_dummies(df_edge_list)
# df_edge_dummies.head()

In [55]:
#prep labels
df_label =df.reset_index()[['id','edge_id','type']].drop_duplicates()
df_label.set_index(['id','edge_id'],inplace = True)
df_label['label'] = (df_label["type"] == 'Car').astype(int)

#add edge_id  as index
df.set_index('edge_id', append=True,inplace = True)
df = df.reorder_levels([0,1,3,2])
df.head()

lat        lon  speed  lon_acc  \
id traj edge_id              time                                        
31 0    954712407_6220792258 0.00  37.97613  23.736383    0.0      0.0   
                             0.04  37.97613  23.736383    0.0      0.0   
                             0.08  37.97613  23.736383    0.0      0.0   
                             0.12  37.97613  23.736383    0.0      0.0   
                             0.16  37.97613  23.736383    0.0      0.0   

                                   lat_acc type  traveled_d  avg_speed  \
id traj edge_id              time                                        
31 0    954712407_6220792258 0.00      0.0  Car      177.77   5.765368   
                             0.04      0.0  Car      177.77   5.765368   
                             0.08      0.0  Car      177.77   5.765368   
                             0.12      0.0  Car      177.77   5.765368   
                             0.16      0.0  Car      177.77   5.765368   

                                   bearing  nearest_edge_start_node  \
id traj edge_id              time                                     
31 0    954712407_6220792258 0.00      0.0                954712407   
                             0.04      0.0                954712407   
                             0.08      0.0                954712407   
                             0.12      0.0                954712407   
                             0.16      0.0                954712407   

                                   nearest_edge_end_node  edge_progress  dir  \
id traj edge_id              time                                              
31 0    954712407_6220792258 0.00             6220792258       0.187819    0   
                             0.04             6220792258       0.187819    0   
                             0.08             6220792258       0.187819    0   
                             0.12             6220792258       0.187819    0   
                             0.16             6220792258       0.187819    0   

                                   time_stamp  edge_progress_intervals  \
id traj edge_id              time                                        
31 0    954712407_6220792258 0.00        0.00                      0.1   
                             0.04        0.04                      0.1   
                             0.08        0.08                      0.1   
                             0.12        0.12                      0.1   
                             0.16        0.16                      0.1   

                                   vehicle_density  avg_surr_speed  
id traj edge_id              time                                   
31 0    954712407_6220792258 0.00                8        2.148487  
                             0.04                8        2.161713  
                             0.08                8        2.174713  
                             0.12                8        2.187762  
                             0.16                8        2.199650

In [56]:
#aggregate function
df_agg = df.groupby(['id','traj','edge_id']).agg({
    'speed': ['mean', 'median', 'std'],
    'lon_acc': ['mean','median', 'std'],
    'lat_acc': ['mean','median', 'std'],
    'bearing':['mean','median','std'],
    'dir':['mean'],
    'vehicle_density':['mean','median','std'],
    'avg_surr_speed':['mean','median','std']
    
})
df_agg.head()

speed                        lon_acc  \
                                    mean    median        std      mean   
id traj edge_id                                                           
31 0    250691827_954712407    19.625741  19.65150   0.162016  0.088869   
        250691847_2511789008   15.508343  15.83780   1.166650 -0.534727   
        250691847_954712407    19.830810  19.87575   0.134977 -0.075097   
        2511789008_7384331753  13.514325  13.80255   0.824164 -0.034114   
        42240105_7384331753     9.896906   6.93710  10.022636 -0.133568   

                                                   lat_acc                     \
                                median       std      mean   median       std   
id traj edge_id                                                                 
31 0    250691827_954712407    0.11560  0.041770 -0.570805 -0.59000  0.071565   
        250691847_2511789008  -0.49510  0.258268 -0.220855 -0.20370  0.152986   
        250691847_954712407    0.01955  0.156025 -0.423980 -0.40275  0.055956   
        2511789008_7384331753 -0.03050  0.365167  0.113490  0.00080  0.267997   
        42240105_7384331753    0.00000  0.659121  0.083461  0.00000  0.257939   

                                bearing                           dir  \
                                   mean    median       std      mean   
id traj edge_id                                                         
31 0    250691827_954712407   -1.606260 -1.570795  0.332595  0.051282   
        250691847_2511789008  -2.506814 -2.808497  0.585137  0.058824   
        250691847_954712407   -1.920152 -1.570795  0.541961  0.000000   
        2511789008_7384331753 -2.509381 -2.808801  0.611348  1.000000   
        42240105_7384331753   -0.717845 -0.332527  0.775122  0.000000   

                              vehicle_density                  avg_surr_speed  \
                                         mean median       std           mean   
id traj edge_id                                                                 
31 0    250691827_954712407          1.358974    1.0  0.485971      19.947077   
        250691847_2511789008         1.000000    1.0  0.000000      15.508343   
        250691847_954712407          1.000000    1.0  0.000000      19.830810   
        2511789008_7384331753        1.253968    1.0  0.437017      13.942884   
        42240105_7384331753          2.907121    3.0  0.910258      13.213937   

                                                    
                                  median       std  
id traj edge_id                                     
31 0    250691827_954712407    19.788800  0.460166  
        250691847_2511789008   15.837800  1.166650  
        250691847_954712407    19.875750  0.134977  
        2511789008_7384331753  13.886600  1.378659  
        42240105_7384331753    10.945113  9.296109

In [59]:
#join features
df_features = df_agg.join(df_label[['type','label']])
df_features.head()

C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(speed, mean)  (speed, median)  (speed, std)  \
id edge_id               traj                                                 
31 250691827_954712407   0         19.625741         19.65150      0.162016   
   250691847_2511789008  0         15.508343         15.83780      1.166650   
   250691847_954712407   0         19.830810         19.87575      0.134977   
   2511789008_7384331753 0         13.514325         13.80255      0.824164   
   42240105_7384331753   0          9.896906          6.93710     10.022636   

                               (lon_acc, mean)  (lon_acc, median)  \
id edge_id               traj                                       
31 250691827_954712407   0            0.088869            0.11560   
   250691847_2511789008  0           -0.534727           -0.49510   
   250691847_954712407   0           -0.075097            0.01955   
   2511789008_7384331753 0           -0.034114           -0.03050   
   42240105_7384331753   0           -0.133568            0.00000   

                               (lon_acc, std)  (lat_acc, mean)  \
id edge_id               traj                                    
31 250691827_954712407   0           0.041770        -0.570805   
   250691847_2511789008  0           0.258268        -0.220855   
   250691847_954712407   0           0.156025        -0.423980   
   2511789008_7384331753 0           0.365167         0.113490   
   42240105_7384331753   0           0.659121         0.083461   

                               (lat_acc, median)  (lat_acc, std)  \
id edge_id               traj                                      
31 250691827_954712407   0              -0.59000        0.071565   
   250691847_2511789008  0              -0.20370        0.152986   
   250691847_954712407   0              -0.40275        0.055956   
   2511789008_7384331753 0               0.00080        0.267997   
   42240105_7384331753   0               0.00000        0.257939   

                               (bearing, mean)  ...  (bearing, std)  \
id edge_id               traj                   ...                   
31 250691827_954712407   0           -1.606260  ...        0.332595   
   250691847_2511789008  0           -2.506814  ...        0.585137   
   250691847_954712407   0           -1.920152  ...        0.541961   
   2511789008_7384331753 0           -2.509381  ...        0.611348   
   42240105_7384331753   0           -0.717845  ...        0.775122   

                               (dir, mean)  (vehicle_density, mean)  \
id edge_id               traj                                         
31 250691827_954712407   0        0.051282                 1.358974   
   250691847_2511789008  0        0.058824                 1.000000   
   250691847_954712407   0        0.000000                 1.000000   
   2511789008_7384331753 0        1.000000                 1.253968   
   42240105_7384331753   0        0.000000                 2.907121   

                               (vehicle_density, median)  \
id edge_id               traj                              
31 250691827_954712407   0                           1.0   
   250691847_2511789008  0                           1.0   
   250691847_954712407   0                           1.0   
   2511789008_7384331753 0                           1.0   
   42240105_7384331753   0                           3.0   

                               (vehicle_density, std)  (avg_surr_speed, mean)  \
id edge_id               traj                                                   
31 250691827_954712407   0                   0.485971               19.947077   
   250691847_2511789008  0                   0.000000               15.508343   
   250691847_954712407   0                   0.000000               19.830810   
   2511789008_7384331753 0                   0.437017               13.942884   
   42240105_7384331753   0                   0.910258               13.213937   

                               (avg_surr_speed, median)  \
id edge_i

In [60]:
#merge id and traj
df_features.index = [df_features.index.get_level_values(1),df_features.index.map('{0[2]}/{0[0]}'.format)]
df_features.index.set_names("traj",inplace = True,level = 1)
df_features.head()

,,"(speed, mean)","(speed, median)","(speed, std)","(lon_acc, mean)","(lon_acc, median)","(lon_acc, std)","(lat_acc, mean)","(lat_acc, median)","(lat_acc, std)","(bearing, mean)",...,"(bearing, std)","(dir, mean)","(vehicle_density, mean)","(vehicle_density, median)","(vehicle_density, std)","(avg_surr_speed, mean)","(avg_surr_speed, median)","(avg_surr_speed, std)",type,label
edge_id,traj,,,,,,,,,,,,,,,,,,,,,
250691827_954712407,0/31,19.625741,19.65150,0.162016,0.088869,0.11560,0.041770,-0.570805,-0.59000,0.071565,-1.606260,...,0.332595,0.051282,1.358974,1.0,0.485971,19.947077,19.788800,0.460166,Car,1
250691847_2511789008,0/31,15.508343,15.83780,1.166650,-0.534727,-0.49510,0.258268,-0.220855,-0.20370,0.152986,-2.506814,...,0.585137,0.058824,1.000000,1.0,0.000000,15.508343,15.837800,1.166650,Car,1
250691847_954712407,0/31,19.830810,19.87575,0.134977,-0.075097,0.01955,0.156025,-0.423980,-0.40275,0.055956,-1.920152,...,0.541961,0.000000,1.000000,1.0,0.000000,19.830810,19.875750,0.134977,Car,1
2511789008_7384331753,0/31,13.514325,13.80255,0.824164,-0.034114,-0.03050,0.365167,0.113490,0.00080,0.267997,-2.509381,...,0.611348,1.000000,1.253968,1.0,0.437017,13.942884,13.886600,1.378659,Car,1
42240105_7384331753,0/31,9.896906,6.93710,10.022636,-0.133568,0.00000,0.659121,0.083461,0.00000,0.257939,-0.717845,...,0.775122,0.000000,2.907121,3.0,0.910258,13.213937,10.945113,9.296109,Car,1


In [61]:
#edges which have both car and taxi
df_label_1 = df_label.reset_index()[["edge_id","type","label"]].drop_duplicates()
both_car_taxi = df_label_1.groupby(['edge_id']).agg({
   'label':'count'    
})
both_car_taxi = both_car_taxi[both_car_taxi['label'] >1]
both_car_taxi.index

Index(['250691668_250691669', '250691668_250698924', '250691668_250709582',
       '250691669_250691670', '250691669_250691724', '250691670_250691723',
       '250691670_250706414', '250691723_250691724', '250691723_250691755',
       '250691723_3339821648', '250691724_250698924', '250691724_653971685',
       '250691794_3339821648', '250691827_2511789008', '250691827_3339821648',
       '250691827_954712407', '250691847_2511789008', '250698924_250698926',
       '2511789008_7384331753', '42240105_250691670', '42240105_7384331753',
       '7384331753_7749831282', '954712407_2511789008', '954712407_6220792258',
       '954712428_7384331753', '954712428_7749831283'],
      dtype='object', name='edge_id')

In [62]:
#get those entries which have both cars and taxis plying in the edge
df_features = df_features.loc[both_car_taxi.index]
df_features.head()

(speed, mean)  (speed, median)  (speed, std)  \
edge_id             traj                                                  
250691668_250691669 0/277      23.927675         21.04300     13.962709   
                    0/286      23.602849         21.79915     10.854241   
                    0/313      25.295323         25.77800      3.872052   
                    0/341      37.678595         37.86355      1.021791   
                    0/345      36.114424         36.02000      0.413763   

                           (lon_acc, mean)  (lon_acc, median)  (lon_acc, std)  \
edge_id             traj                                                        
250691668_250691669 0/277        -0.533191           -0.39780        1.596330   
                    0/286        -0.513847           -0.67210        1.255232   
                    0/313        -0.220269           -0.28765        0.665402   
                    0/341         0.239245            0.10825        0.292089   
                    0/345         0.015700            0.04945        0.304796   

                           (lat_acc, mean)  (lat_acc, median)  (lat_acc, std)  \
edge_id             traj                                                        
250691668_250691669 0/277         0.010792            0.00560        0.141282   
                    0/286         0.003515           -0.01030        0.095329   
                    0/313         0.011152            0.02025        0.219332   
                    0/341         0.084988            0.07125        0.167702   
                    0/345         0.013152           -0.00460        0.284540   

                           (bearing, mean)  ...  (bearing, std)  (dir, mean)  \
edge_id             traj                    ...                                
250691668_250691669 0/277         2.751820  ...        0.742794     0.948339   
                    0/286         2.962455  ...        0.254307     0.996350   
                    0/313         2.989037  ...        0.051978     1.000000   
                    0/341         2.995736  ...        0.031443     1.000000   
                    0/345         2.997782  ...        0.031417     1.000000   

                           (vehicle_density, mean)  (vehicle_density, median)  \
edge_id             traj                                                        
250691668_250691669 0/277                 2.114391                        2.0   
                    0/286                 1.660584                        1.0   
                    0/313                 1.796875                        2.0   
                    0/341                 2.098837                        2.0   
                    0/345                 1.853933                        2.0   

                           (vehicle_density, std)  (avg_surr_speed, mean)  \
edge_id             traj                                                    
250691668_250691669 0/277                1.053133               26.830473   
                    0/286                0.773732               28.076568   
                    0/313                0.712459               30.861133   
                    0/341                0.672430               37.297653   
                    0/345                0.796185               35.432054   

                           (avg_surr_speed, median)  (avg_surr_speed, std)  \
edge_id             traj                                                     
250691668_250691669 0/277                 26.071900              11.184630   
                    0/286                 27.269400               8.006914   
                    0/313                 32.746725               5.757262   
                    0/341                 36.169767               3.617100   
                    0/345                 35.755500               2.361569   

                           type label  
edge_id             traj               
250691668_250691669 0/277  Taxi     0  
                    0/286   C

In [96]:
#count of how many taxis and cars ply in an edge
#count_list = df_features.groupby('edge_id').count()['type']
count_list = df_features.reset_index('edge_id').pivot_table(index = 'edge_id',values = 'label',columns = 'type',aggfunc = ['count'])

In [106]:
#creating a mask to get equal numbers of cars and taxis in test set
df_mask = df_features.reset_index().set_index(['edge_id','traj','type']).groupby(['edge_id','type']).cumcount().to_frame()+1
df_mask = df_mask.join(count_list,on=['edge_id'])
test_limit = 0.9
df_mask["min_vehicle"] = [df_mask.iloc[i,2] if df_mask.iloc[i,1]>df_mask.iloc[i,2] else df_mask.iloc[i,1] for i in range(0,df_mask.shape[0])]

0  (count, Car)  (count, Taxi)  min_vehicle
edge_id              traj  type                                              
250691668_250691669  0/277 Taxi   1            12              6            6
                     0/286 Car    1            12              6            6
                     0/313 Car    2            12              6            6
                     0/341 Taxi   2            12              6            6
                     0/345 Taxi   3            12              6            6
...                              ..           ...            ...          ...
954712428_7749831283 0/371 Taxi   2            11              3            3
                     0/457 Taxi   3            11              3            3
                     0/473 Car    9            11              3            3
                     0/484 Car   10            11              3            3
                     0/490 Car   11            11              3            3

[788 rows x 4 columns]

In [109]:
df_test_mask =df_mask[df_mask[0]>np.floor(test_limit*df_mask['min_vehicle'])]
df_test_mask = pd.MultiIndex.from_tuples(list(zip(df_test_mask.index.get_level_values(0),df_test_mask.index.get_level_values(1))),names = ['edge_id','traj'])
df_train_mask = df_mask[df_mask[0]<=np.floor(test_limit*df_mask['min_vehicle'])]
df_train_mask = pd.MultiIndex.from_tuples(list(zip(df_train_mask.index.get_level_values(0),df_train_mask.index.get_level_values(1))),names = ['edge_id','traj'])

In [111]:
#generate test set
df_test = df_features[df_features.index.isin(df_test_mask)]
df_test.shape

(400, 21)

In [112]:
#generate training set
df_train = df_features[df_features.index.isin(df_train_mask)]
df_train.shape

(388, 21)

In [113]:
df_train = df_train.dropna()
df_train_x = df_train.copy()
df_train_x.drop(['type','label'],axis = 1,inplace = True)
df_train_y = df_train['label']

In [114]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma = 'auto'))
clf.fit(df_train_x, df_train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [115]:
df_test = df_test.dropna()
df_test_x = df_test.copy()
df_test_x.drop(['type','label'],axis = 1,inplace = True)
df_test_y = df_test['label']

In [116]:
clf.score(df_test_x,df_test_y)

0.3975

In [117]:
#naive prediction predict everything as 1
(df_test_y == 1).astype(int).mean()

0.89

In [118]:
clf.predict(df_test_x)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,

In [119]:
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state=0).fit(df_train_x, df_train_y)  
clf_log.score(df_test_x,df_test_y)

C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.445

In [120]:
clf_log.predict(df_test_x)

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,

In [121]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(df_train_x,df_train_y)

KNeighborsClassifier(n_neighbors=3)

In [124]:
model.score(df_test_x,df_test_y)

0.4475

In [123]:
from sklearn.manifold import TSNE

In [78]:
tsne = TSNE(n_components=2, init='random',
                         random_state=0, perplexity=30)
Y = tsne.fit_transform(df_train_x)

In [80]:
Y_df = pd.DataFrame(data = Y,columns = ['X','Y'])
df_train_y = pd.concat([df_train_y.reset_index(),Y_df] , axis = 1)


In [82]:
import matplotlib.pyplot as plt
plt.scatter(df_train_y['X'],df_train_y['Y'])

,edge_id,traj,label,X,Y
0,250691668_250691669,0/313,1,-12.296868,28.329298
1,250691668_250691669,0/341,0,0.099054,35.857887
2,250691668_250691669,0/345,0,-1.080303,34.812042
3,250691668_250691669,0/351,1,-6.678246,31.108419
4,250691668_250691669,0/354,0,4.365138,37.408707
...,...,...,...,...,...
729,954712428_7749831283,0/371,0,-2.776900,-17.199165
730,954712428_7749831283,0/457,0,-14.449141,-2.574192
731,954712428_7749831283,0/473,1,-9.131047,-12.012199
732,954712428_7749831283,0/484,1,-9.201050,-13.171971
